In [ ]:
from astropy.io import ascii, fits
from astropy.table import Table, vstack
import numpy as np
import matplotlib.pyplot as plt
import es_gen
import pdb
import glob
from scipy.interpolate import interp1d
from astropy.convolution import Gaussian1DKernel, convolve
import os
import es_gen
import sp_engine
from sp_engine import measuredArray
from specutils.io import read_fits

%matplotlib inline

Inspecting the data from Masuda

In [ ]:
dat = Table(fits.getdata('kic1255_fits/w111346.fits'))
dat.sort('Wavelength')
rvSystematic = -36.26 ## Croll et al. 2014 systematic RV + my offset to match Mg I lines
rvBary = -2.9142 ## barycentric correction
dat['Wavelength'] = dat['Wavelength'] / (1. + (rvSystematic - rvBary)/3e5)

In [ ]:
dat2 = Table(fits.getdata('kic1255_fits/111346.fits'))
dat2.sort('Wavelength')
dat2['Wavelength'] = dat2['Wavelength'] / (1. + (rvSystematic - rvBary)/3e5) ## needed to match Mg lines

## Plotting the Masuda et al. Reduction

In [ ]:
fig, ax = plt.subplots()
waveOffset = 0.5
ax.plot(dat['Wavelength'],dat['Flux']/300.+ 1.5,label='w111346.txt')
ax.plot(dat2['Wavelength'],dat2['Flux'],label='111346.txt')
ax.legend()
ax.set_xlabel('Wavelength ($\AA$)')
ax.set_ylabel('Normalized Flux + Offset')
ax.set_xlim(5163,5190)

ax.set_ylim(-2,5)

ax.axvline(5167.3216)
ax.axvline(5172.6843)
ax.axvline(5183.6042)
fig.savefig('mgTriplet.pdf')


We want the files like w111346.txt (dat) and not 11346 (dat2) since the normalization process for RV wipes out the Mg lines.

In [ ]:
fig, ax = plt.subplots()
ax.plot(dat['Wavelength'],dat['RawFlux'])
ax.plot(dat['Wavelength'],dat['Flux'])
ax.legend()
ax.set_xlim(5163,5190)
ax.set_ylim(0,1000)

## Look at some BOSZ models

In [ ]:
HDUList = fits.open('../bosz/amp00cp00op00t4500g40v20modrt0b100000rs.fits')
model = Table(HDUList[1].data)
model['Wavelength'] = model['Wavelength'] * (1. - 81.2/3e5) ## why do we need this RV offset??
HDUList.close()

In [ ]:
#model.colnames
#HDUList.info()
#HDUList[1].header
#head = HDUList[0].header
#head['LOGG']

Look at the Mg lines

In [ ]:
fig, ax = plt.subplots()
pts = (model['Wavelength'] > 5176) & (model['Wavelength'] < 5180)
norm = np.nanmedian(model['SpecificIntensity'][pts])
ax.plot(model['Wavelength'],model['SpecificIntensity']/norm)
ax.set_xlim(5163,5190)
#ax.set_xlim(5172,5174)
ax.axvline(5167.3216,color='red')
ax.axvline(5172.6843,color='red')
ax.axvline(5183.6042,color='red')

Look at the Na I D lines

In [ ]:
plt.plot(model['Wavelength'],model['SpecificIntensity']/norm)
plt.xlim(5880,5910)
plt.axvline(5889.95095,color='red')
plt.axvline(5895.92424,color='red')

In [ ]:
HDUList = fits.open('../bosz/amp00cp00op00t4500g40v20modrt0b100000rs.fits')
model = Table(HDUList[1].data)
model.colnames
norm = np.nanmedian(model['SpecificIntensity'][pts])
model['Wavelength'] = model['Wavelength'] * (1. - 0./3e5) ## why no RV offset for the IR?
HDUList.close()

plt.plot(model['Wavelength'],model['SpecificIntensity']/norm)
plt.xlim(22000,22200)
plt.ylim(0,0.25)
plt.axvline(22062.42,color='red')
plt.axvline(22089.69,color='red')


Read in all the relevant BOSZ models

In [ ]:
m1 = sp_engine.stellModel('../bosz/amp00cp00op00t4500g40v20modrt0b100000rs.fits')
m2 = sp_engine.stellModel('../bosz/amp00cp00op00t4500g45v20modrt0b100000rs.fits')
m3 = sp_engine.stellModel('../bosz/amp00cp00op00t5000g40v20modrt0b100000rs.fits')
m4 = sp_engine.stellModel('../bosz/amp00cp00op00t4500g50v20modrt0b100000rs.fits')
m5 = sp_engine.stellModel('../bosz/amp00cp00op00t4500g30v20modrt0b100000rs.fits')
m6 = sp_engine.stellModel('../bosz/amp00cp00op00t4000g45v20modrt0b100000rs.fits')
m7 = sp_engine.stellModel('../bosz/amp00cp00op00t5000g45v20modrt0b100000rs.fits')
m8 = sp_engine.stellModel('../bosz/amp05cp00op00t4500g45v20modrt0b100000rs.fits')
m9 = sp_engine.stellModel('../bosz/amm05cp00op00t4500g45v20modrt0b100000rs.fits')
m10 = sp_engine.stellModel('../bosz/amp00cm05op00t4500g45v20modrt0b100000rs.fits')
m11 = sp_engine.stellModel('../bosz/amp00cp05op00t4500g45v20modrt0b100000rs.fits')
m12 = sp_engine.stellModel('../bosz/amp00cp00om03t4500g45v20modrt0b100000rs.fits')
m13 = sp_engine.stellModel('../bosz/amp00cp00op03t4500g45v20modrt0b100000rs.fits')
#m14 = sp_engine.stellModel('../bosz/amp00cp00om03t4500g40v20modrt0b100000rs.fits')

In [ ]:
fig, ax = plt.subplots()
showRange1 = [5160,5195]
normRange1 = [5188,5190]
m4.plot(ax=ax,showRange=showRange1,normRange=normRange1)
m5.plot(ax=ax,offset=0.,showRange=showRange1,normRange=normRange1)
#m3.plot(ax=ax,offset=0.)
ax.set_xlim(showRange1)
ax.legend()
ax.axvline(5167.3216)
ax.axvline(5172.6843)
ax.axvline(5183.6042)

## Compare data to models

In [ ]:
## Normalize

closepts = (dat['Wavelength'] > 5160.) & (dat['Wavelength'] < 5195.)
normValue1 = np.median(dat['Flux'][closepts])
pts = closepts & (dat['Flux'] > normValue1 * 1.1)
poly = es_gen.robust_poly(dat['Wavelength'][pts],dat['Flux'][pts],4)
linModel = np.polyval(poly,dat['Wavelength'])
detrFlux = dat['Flux'] / linModel

In [ ]:
plt.plot(dat['Wavelength'],detrFlux)
#plt.plot(dat['Wavelength'][pts],detrFlux[pts])
plt.xlim(5160,5195)
plt.ylim(0,2)
plt.axvline(5167.3216,color='red')
plt.axvline(5172.6843,color='red')
plt.axvline(5183.6042,color='red')

In [ ]:
# fig, ax = plt.subplots()
# m1.plot(ax=ax)
# m2.plot(ax=ax,offset=0.)
# ax.plot(dat['Wavelength'] + 2.,detrFlux,label='w111346.fits')
# ax.set_xlim(5163,5190)
# ax.set_ylim(-0.1,2)
# ax.set_xlabel('Wavelength ($\AA$)')
# ax.set_ylabel('Normalized Flux')
# ax.legend()

## Convolve the models from R=100,000 to data at R=60,000
Gaussian $\sigma = \frac{1}{\sqrt{(1./6e4)^2 - (1./1e5)^2}}$ = 75,000

In [ ]:
refWavel = 5163.0 ## angstroms
m1.smooth(refWavel,6e4)
m2.smooth(refWavel,6e4)
m3.smooth(refWavel,6e4)
m1.RConv

Look closely to check convolution

In [ ]:
fig, ax = plt.subplots()

m1.plot(ax=ax,doSmooth=True)
m3.plot(ax=ax,offset=0.,doSmooth=True)
ax.plot(dat['Wavelength'],detrFlux,label='w111346.fits')
ax.set_xlim(5170,5173)
ax.set_ylim(-0.1,2)
ax.set_xlabel('Wavelength ($\AA$)')
ax.set_ylabel('Normalized Flux')
ax.legend()

Tweak the convolution

In [ ]:
#m1.smooth(refWavel,5.5e4)
m1.smooth(refWavel,8.5e4)
m2.smooth(refWavel,8.5e4)
m3.smooth(refWavel,8.5e4)
fig, ax = plt.subplots()
m1.plot(ax=ax,doSmooth=True)
m2.plot(ax=ax,offset=0.,doSmooth=True)
ax.plot(dat['Wavelength'],detrFlux,label='w111346.fits')
ax.set_xlim(5170,5174)
ax.set_ylim(-0.1,2)
ax.set_xlabel('Wavelength ($\AA$)')
ax.set_ylabel('Normalized Flux')
ax.legend()

For some reason, it matches better if we convolved with a smaller kernel width

In [ ]:
5172./m1.RConv, m1.RConv

In [ ]:
fig, ax = plt.subplots()

#m1.plot(ax=ax,offset=0.7,doSmooth=True)
m2.plot(ax=ax,offset=0.7,doSmooth=True)
m3.plot(ax=ax,offset=-0.7,doSmooth=True)

ax.plot(dat['Wavelength'],detrFlux,label='w111346.fits')
ax.set_xlim(5163,5190)
ax.set_ylim(-1,3)
ax.set_xlabel('Wavelength ($\AA$)')
ax.set_ylabel('Normalized Flux')
ax.legend()
fig.savefig('bosz_mg_triplet.pdf')

## Compare two different images

In [ ]:
s1 = sp_engine.subaruSpec('kic1255_fits/w111346.fits',rvCorrection=rvSystematic - -2.9142)
s2 = sp_engine.subaruSpec('kic1255_fits/w111350.fits',rvCorrection=rvSystematic - -2.936)
#s2 = sp_engine.subaruSpec('kic1255_fits/w111526.fits')

## Check that the sp_engine.subaruSpec class works

In [ ]:
fig, ax = plt.subplots()
ax.plot(dat['Wavelength'],detrFlux,label='w111346.fits, detrend')
ax.plot(dat['Wavelength'],dat['Flux']/300.,label='w111346.fits')
ax.plot(s1.wave,s1.normFlux * 0.8,label='s1 object')
#s1.plot(ax=ax)
ax.legend()
ax.set_xlim(5170,5174)
#ax.set_xlim(5160,5190)
ax.set_ylim(0,2)

# ax.axvline(5167.3216)
# ax.axvline(5172.6843)
# ax.axvline(5183.6042)

In [ ]:
len(s2.dat['Wavelength']), len(s2.normFlux), len(s1.dat['Wavelength'])

In [ ]:
fig, ax = plt.subplots()
s1.plot(ax=ax)
s2.plot(ax=ax)
ax.legend()
ax.set_xlabel('Wavelength ($\AA$)')
ax.set_ylabel('Normalized Flux + Offset')
#ax.set_xlim(5163,5190)
#ax.set_xlim(5163,5170)
ax.set_xlim(5166.5,5167)
ax.set_ylim(0,2)

Differences between spectra are much smaller than the differences with the BOSZ models

## Average all Masuda Spectra

In [ ]:
np.min(s2.dat['Wavelength']), np.max(s2.dat['Wavelength'])

In [ ]:
baryCorr = ascii.read('output/barycent_corrections.csv')

In [ ]:
fList = glob.glob('kic1255_fits/w*.fits')
blueCCDList, allSubaru = [], []
for oneFile in fList:
    s = sp_engine.subaruSpec(oneFile)
    if s.ccdSide == 'blue':
        blueCCDList.append(oneFile)

nFile = len(blueCCDList)
for ind,oneFile in enumerate(blueCCDList):
    baryInd = baryCorr['File Name'] == os.path.basename(oneFile)
    if np.sum(baryInd) != 1:
        print("Couldn't find RV correctly for file {}".format(oneFile))
    RV = baryCorr['RV (km/s)'][baryInd]
    s = sp_engine.subaruSpec(oneFile,rvCorrection=rvSystematic - RV)
    
    if ind == 0:
        ## Choose the wavelength grid the first time through
        pts = (s.wave > 5120.) & (s.wave < 6440.)
        waveGrid = s.wave[pts]
        allFlux = np.zeros([len(waveGrid),nFile])
    f = interp1d(s.wave,s.normFlux)
    allFlux[:,ind] = f(waveGrid)
    allSubaru.append(s)

Also apply the RVs to s1 and s2

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
for ind,oneS in enumerate(allSubaru):
    #ax.plot(oneS.wave,oneS.normFlux + 0.3 * ind)
    ax.plot(waveGrid,allFlux[:,ind] + 0.3 * ind)
ax.set_xlim(5170,5174)
#ax.set_xlim(5173,5176)
#ax.set_xlim(5160,5190)
ax.set_ylim(0,4.)
#ax.set_ylim(0,5.)


In [ ]:
#pts = (waveGrid > 5160.) & (waveGrid < 5190.)
pts = (waveGrid > 5170.) & (waveGrid < 5174.)
fig, ax =plt.subplots(figsize=(14,5))
plt.imshow(allFlux[pts,:].T,aspect='auto')

In [ ]:
medSpec = np.median(allFlux,axis=1)
medSpecObj = measuredArray(waveGrid,medSpec,Res=6e4)
medSpecObj.removePoly()
## Get Errors from the standard deviation
stdSpec = np.nanstd(allFlux,axis=1)
medSNR = np.median(medSpec/stdSpec)
medSNR

In [ ]:
stdeMean = stdSpec/ np.sqrt(nFile)
## mask points that have extraordinarily high errors
badp = stdeMean > 0.9
stdeMean[badp] = np.nan

Compare median spectra with an example one

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
thisRange = (5170,5174)
pts = (waveGrid > thisRange[0]) & (waveGrid < thisRange[1])
ax.plot(waveGrid[pts],medSpec[pts],label='median')
ax.errorbar(waveGrid[pts],np.ones_like(waveGrid[pts]) *0.1,fmt='.',yerr=stdSpec[pts],
           label='Error on Single Spec')
s1.plot(ax=ax)
s2.plot(ax=ax)
ax.legend()
ax.set_xlim(thisRange)
ax.set_ylim(0,1.5)

In [ ]:
fig, ax = plt.subplots()
ax.plot(waveGrid[pts],medSpec[pts],label='median')
s1.plot(ax=ax)
s2.plot(ax=ax)
ax.legend()
ax.set_xlim(5171.7,5172.3)
ax.set_ylim(0,1.5)

## Compare 2015 Data to 2013 Data

In [ ]:
specES = read_fits.read_fits_spectrum1d('../fits_data/sp_ecw_HDSA00094086_improved_wavecal.fits')
oneES= specES[22]
waveES13, fluxES13 = oneES.wavelength.value,oneES.flux.value
waveES13 = waveES13 / (1. + (rvSystematic - 6.5924)/3.0e5)
badp = (fluxES13 > 2. * np.median(fluxES13))
fluxES13[badp] = np.nan
spec13Obj = measuredArray(waveES13,fluxES13)
spec13Obj.removePoly()

In [ ]:
fig, ax = plt.subplots(figsize=(15,4))
normRange1 = [5176,5179]
medSpecObj.plot(ax=ax,customName='2015 spec',normRange=normRange1,removePoly=False)
#spec13Obj.plot(ax=ax,customName='2013 spec',normRange=normRange1,removePoly=True)
ax.plot(waveES13,(fluxES13/np.nanmedian(fluxES13) - 1.)*1.2+1.,label='2013 spec')
ax.legend()
ax.set_xlabel('Wavelength ($\AA$)')
ax.set_ylabel('Normalized Flux')
ax.set_ylim(0,2)
ax.set_xlim(5160,5190)
fig.savefig('plots/2013_vs_2015_spec.pdf',bbox_inches='tight')

# Adding some offsets to check for differences

In [ ]:
fig, ax = plt.subplots(figsize=(15,4))
normRange1 = [5176,5179]
medSpecObj.plot(ax=ax,customName='2015 spec',normRange=normRange1,removePoly=False)
#spec13Obj.plot(ax=ax,customName='2013 spec',normRange=normRange1,removePoly=True)
ax.plot(waveES13,(fluxES13/np.nanmedian(fluxES13) - 1.)*1.2+0.6,label='2013 spec')
ax.legend()
ax.set_xlabel('Wavelength ($\AA$)')
ax.set_ylabel('Normalized Flux')
ax.set_ylim(0,2)
ax.set_xlim(5160,5190)

## Compare Median Spectrum with BOSZ Model

In [ ]:
medSpecObj.smooth(5715,outRes=4.0e4)
for oneMod in [m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13]:
    oneMod.removePoly()
    #oneMod.smooth(5715,outRes=6.0e4)
    oneMod.smooth(5715,outRes=8.5e4)


In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
showRange = (5163,5190)
normRange1 = [5176,5179]
#m1.plot(ax=ax,offset=0.7,doSmooth=True,removePoly=False)
m2.plot(ax=ax,offset=0.7,doSmooth=True,removePoly=False,normRange=normRange1)
m3.plot(ax=ax,offset=-0.7,doSmooth=True,removePoly=False,normRange=normRange1)
#ax.plot(dat['Wavelength'] + 2.,detrFlux,label='w111346.fits')
#ax.plot(waveGrid+2.,medSpec+0.,label='median_spec')
medSpecObj.plot(ax=ax,customName='median spec',removePoly=False,doSmooth=False,
                normRange=normRange1)
ax.set_xlim(showRange)
ax.set_ylim(-1,4.)
ax.set_xlabel('Wavelength ($\AA$)')
ax.set_ylabel('Normalized Flux')

nGrid = len(waveGrid)
#skipPoints = np.mod(np.arange(nGrid),30) == 0 ## skip points to better see the errors
pts = (waveGrid > showRange[0]) & (waveGrid < showRange[1]) #& skipPoints

ax.errorbar(waveGrid[pts],np.zeros_like(waveGrid[pts]) - 0.8,fmt='.',yerr=stdeMean[pts],
           label='Standard err')
ax.legend()

fig.savefig('bosz_mg_triplet_median_spec_w_5000.pdf')

Calculate the line width from microturbulent velocity in BOSZ models

In [ ]:
lambdaRef = 5312.85#5312.28
v_over_c = 2.0 / 3e5
lambdaDiff = v_over_c * lambdaRef
normRangeLineCheck = [5312.5,5313.4]
fig, ax = plt.subplots()
m2.plot(ax=ax,offset=0.7,doSmooth=True,normRange=normRangeLineCheck,showRange=normRangeLineCheck)

medSpecObj.plot(ax=ax,customName='median spec',doSmooth=False,normRange=normRangeLineCheck,
                showRange=normRangeLineCheck)


ax.axvline(lambdaRef,color='purple',linewidth=0.5)
ax.axvline(lambdaRef + lambdaDiff,color='purple',linewidth=0.5)

ax.axvline(lambdaRef + 7./3.0e5 * lambdaRef,color='red')


The line width appears a lot broader than 2km/s. The red line is 7 km/s away.

In [ ]:
fig, ax = plt.subplots()

normRange1 = [5306,5328]
showRange2 = normRange1
#m1.plot(ax=ax,offset=0.7,doSmooth=True,removePoly=False)
m2.plot(ax=ax,offset=0.7,doSmooth=False,removePoly=False,normRange=normRange1,
       showRange=showRange2)
m3.plot(ax=ax,offset=-0.7,doSmooth=False,removePoly=False,normRange=normRange1,
       showRange=showRange2)

#ax.plot(waveGrid+2.,medSpec+0.,label='median_spec')
medSpecObj.plot(ax=ax,customName='median spec',removePoly=False,doSmooth=False,
                normRange=normRange1,showRange=showRange2)
ax.set_xlim(normRange1[0],normRange1[1])
ax.set_ylim(-1,3)
ax.set_xlabel('Wavelength ($\AA$)')
ax.set_ylabel('Normalized Flux')

pts = (waveGrid > 5305) & (waveGrid < 5330)

ax.errorbar(waveGrid[pts],np.zeros_like(waveGrid[pts]) - 0.8,fmt='.',yerr=stdeMean[pts],
           label='Standard err')
ax.legend()
fig.savefig('plots/continuum_region.pdf')

In [ ]:
plt.plot(waveGrid,medSpec/stdeMean,'.')
plt.ylim(0,100)
plt.xlim(5163,5190)

## Zoom in on Blue Side

In [ ]:
normRange1 = [5166,5170]
fig, ax = plt.subplots()
for oneMod in [m1,m3]:
    oneMod.plot(ax=ax,offset=0,doSmooth=True,removePoly=False,normRange=normRange1)

medSpecObj.plot(ax=ax,customName='median spec',removePoly=False,normRange=normRange1,doSmooth=True)
ax.set_xlim(5166,5170)
ax.set_ylim(-0.2,5)
ax.set_xlabel('Wavelength ($\AA$)')
ax.set_ylabel('Normalized Flux')
ax.legend()


In [ ]:
medSpecObj = measuredArray(waveGrid,medSpec,Res=6e4)
medSpecObj.removePoly()
medSpecObj.smooth(5715,outRes=4.0e4)


In [ ]:
paramList = ['T_EFF','LOGG','MH','CM','ALPHA']
paramDict = {'T_EFF':'T','LOGG':'logg','MH':'[M/H]','CM':'[C/M]','ALPHA':r'[$\alpha$/M]'}

styleList=['-','--',':']

def paramLabel(parameter,model):
    """ Returns a string of the name = value"""
    if parameter == 'FINAL':
        return 'Best Fit'
    else:
        return paramDict[parameter]+'='+str(model.header[parameter])

def doExploration(modelList,parameter='T_EFF',xlimExplore=(5163,5190)):
    """ Does an exploration of a parameter 
    Parameters
    ----------------
    modelList: list of model objects
        List of models to show exploration over
    parameter: str
        FITS keyword of parameter to show in legend
    xlimExplore: tuple
        X limits fo plot
    """
    polyRange = [5163,5190]
    normRange1 = [5176,5179]
    fig, ax = plt.subplots()
    medSpecObj.removePoly(waveRange=polyRange,nPoly=3,sigreject=10)
    medSpecX, medSpecY = medSpecObj.getSpec(removePoly=True,normRange=normRange1,
                                            doSmooth=False)

    
    for ind, oneMod in enumerate(modelList):
        oneMod.removePoly(waveRange=polyRange,nPoly=3,sigreject=10)
        
        chiSquare, chiString = medSpecObj.chiSquared(oneMod,normRange=normRange1,SNR=medSNR)
        
        oneMod.plot(ax=ax,offset=ind *0.00,doSmooth=True,removePoly=True,normRange=normRange1,
                   customName=paramLabel(parameter,oneMod)+chiString,
                   linewidth=(3.0 - ind/2.),linestyle=styleList[ind])
        if ind == 0:
            titleList = []
            for oneParam in paramList:
                if (oneParam != parameter) | (parameter == 'FINAL'):
                    titleList.append(paramLabel(oneParam,oneMod))
            ax.set_title(','.join(titleList))
        
        
        
    medSpecObj.plot(ax=ax,customName='median spec',removePoly=True,normRange=normRange1,doSmooth=True,
                    color='C3')
    ax.set_xlim(xlimExplore)
    ax.set_ylim(0,2)
    ax.set_xlabel('Wavelength ($\AA$)')
    ax.set_ylabel('Normalized Flux')
    ax.legend()
    fig.savefig('plots/'+parameter+'_exploration.pdf')

Smooth for plotting purposes

In [ ]:
medSpecObj.smooth(5715,outRes=4.0e4)
for oneMod in [m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13]:
    oneMod.removePoly()
    oneMod.smooth(5715,outRes=4.0e4)
    #oneMod.smooth(5715,outRes=8.5e4)

## Temperature Exploration


In [ ]:
doExploration([m6,m2,m7])

## Gravity Exploration

In [ ]:
doExploration([m5,m1,m4],parameter='LOGG')

## Metallicity Exploration

In [ ]:
doExploration([m8,m2,m9],parameter='MH')

## Carbon Exploration

In [ ]:
doExploration([m10,m2,m11],parameter='CM')


## $\alpha$ Exploration

In [ ]:
doExploration([m12,m2,m13],parameter='ALPHA')



## Find Minimum $\chi^2$ within Grid

In [ ]:
fList2 = glob.glob('../bosz/chi_min_search/*.fits')[0:6]

In [ ]:
def minSearch(fileList):
    """ Does an exploration chi^2 """
    polyRange = [5163,5190]
    normRange1 = [5176,5179]
    
    medSpecObj.removePoly(waveRange=polyRange,nPoly=3,sigreject=10)
    
    allResults =  Table()
    
    for oneMFile in fList2:
        oneResult = Table()
        
        oneMod = sp_engine.stellModel(oneMFile)
        oneMod.smooth(5715,outRes=4.5e4)
        oneMod.removePoly(waveRange=polyRange,nPoly=3,sigreject=10)
        
        chiSquare, chiString = medSpecObj.chiSquared(oneMod,normRange=normRange1,SNR=medSNR)
        
        oneResult['Chi2'] = [chiSquare]
        
        for oneParam in paramList:
            oneResult[oneParam] = [oneMod.header[oneParam]]
        
        oneResult['MName'] = [os.path.basename(oneMFile)]

        allResults = vstack([allResults,oneResult])
    
    return allResults

In [ ]:
minTab = minSearch(fList2)
#minTab

In [ ]:
plt.plot(minTab['T_EFF'],minTab['Chi2'],'o')
plt.ylim(0,1e5)

In [ ]:
minTab[np.argmin(minTab['Chi2'])]

In [ ]:
minTab

## Best Fit

In [ ]:
doExploration([m12],parameter='FINAL')

## Save the spectrum for Spect-Match Emp or Johanna's Tool

In [ ]:
medSpecObj.smooth(5715,outRes=5.8e4)
outDat=Table()
outDat['Wavelength (A)'] = waveGrid
outDat['Flux (Norm)'] = medSpec
outDat['Flux (Smooth)'] = medSpecObj.yconv
outDat['Err'] = stdSpec
outDat.write('output/medianSpec_masuda.csv',overwrite=True)
#waveGrid,medSpec